# STS Benchmark Datasets

For more information, see the implementation of https://github.com/kawine/usif

In [1]:
%load_ext autotime
import numpy as np
import logging
import pandas as pd
import pathlib

from gensim.models import Word2Vec, FastText
from gensim.models.keyedvectors import KeyedVectors, FastTextKeyedVectors

from fse.models.average import Average
from fse.models.sif import SIF
from fse.models.usif import uSIF
from fse.models.inputs import IndexedSentence

from re import sub

from scipy.stats import pearsonr

logging.basicConfig(format='%(asctime)s : %(threadName)s : %(levelname)s : %(message)s',
                    level=logging.INFO
                   )

file_path = "data/stsbenchmark/sts-test.csv"

def normalize_text(sentence):
    return [sub("[^a-zA-Z]", "", w.lower()) for w in sentence.split()] 

def compute_similarities(task_length, model):
    sims = []
    for i, j in zip(range(task_length), range(task_length, 2*task_length)):
        sims.append(model.sv.similarity(i,j))
    return sims

sts_data = pd.read_csv(file_path, sep=",", error_bad_lines=True, names=["cpt","1","task","3","sim","sent_a","sent_b"])
sts_data.dropna(inplace=True)
sts_data.sent_a = sts_data.sent_a.apply(normalize_text)
sts_data.sent_b = sts_data.sent_b.apply(normalize_text)
sts_data.task = sts_data.task.str.replace("2012test", "2012")
sts_data.task = sts_data.task.str.replace("2012train", "None")
sts_data = sts_data[sts_data.task != "None"]

tasks = sorted(["2012", "2013", "2014", "2015", "2016", "2017"])

In [2]:
w2v = KeyedVectors.load("/Users/oliverborchers/Desktop/GSDEV/Models/Static/google_news.model")
ft = FastTextKeyedVectors.load("/Users/oliverborchers/Desktop/GSDEV/Models/Static/ft_crawl_300d_2m.model")

2019-08-19 23:48:27,145 : MainThread : INFO : loading Word2VecKeyedVectors object from /Users/oliverborchers/Desktop/GSDEV/Models/Static/google_news.model
2019-08-19 23:48:33,053 : MainThread : INFO : loading vectors from /Users/oliverborchers/Desktop/GSDEV/Models/Static/google_news.model.vectors.npy with mmap=None
2019-08-19 23:48:35,252 : MainThread : INFO : setting ignored attribute vectors_norm to None
2019-08-19 23:48:35,253 : MainThread : INFO : loaded /Users/oliverborchers/Desktop/GSDEV/Models/Static/google_news.model
2019-08-19 23:48:35,254 : MainThread : INFO : loading FastTextKeyedVectors object from /Users/oliverborchers/Desktop/GSDEV/Models/Static/ft_crawl_300d_2m.model
2019-08-19 23:48:39,011 : MainThread : INFO : loading vectors from /Users/oliverborchers/Desktop/GSDEV/Models/Static/ft_crawl_300d_2m.model.vectors.npy with mmap=None
2019-08-19 23:48:41,748 : MainThread : INFO : loading vectors_vocab from /Users/oliverborchers/Desktop/GSDEV/Models/Static/ft_crawl_300d_2m.mo

time: 20.2 s


In [3]:
w2v_avg = Average(w2v, wv_mapfile_path = "data/vectors/w2v", lang_freq="en")
ft_avg = Average(ft, wv_mapfile_path = "data/vectors/ft", lang_freq="en")

/Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/models/base_s2v.py:157: UserWarning: C extension not loaded, training/inferring will be slow. Install a C compiler and reinstall fse.
  "C extension not loaded, training/inferring will be slow. "
2019-08-19 23:48:47,439 : MainThread : INFO : loading pre-existing wv from /Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/data/vectors/w2v_wv.vectors
2019-08-19 23:48:48,006 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en
2019-08-19 23:48:51,999 : MainThread : INFO : loading pre-existing wv from /Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/data/vectors/ft_wv.vectors
2019-08-19 23:48:52,293 : MainThread : INFO : loading pre-existing vocab from data/vectors/ft_vocab.vectors
2019-08-19 23:48:52,653 : MainThread : INF

time: 8.06 s


In [4]:
models, results = {}, {}

for year in tasks:
    models[f"CBOW-W2V-{year}"] = Average(w2v, wv_mapfile_path = "data/vectors/w2v")
    models[f"SIF-W2V-{year}"] = SIF(w2v, wv_mapfile_path = "data/vectors/w2v")
    models[f"uSIF-W2V-{year}"] = uSIF(w2v, wv_mapfile_path = "data/vectors/w2v")
    
    models[f"CBOW-FT-{year}"] = Average(ft, wv_mapfile_path = "data/vectors/ft")
    models[f"SIF-FT-{year}"] = SIF(ft, wv_mapfile_path = "data/vectors/ft")
    models[f"uSIF-FT-{year}"] = uSIF(ft, wv_mapfile_path = "data/vectors/ft")
    
for k in models:
    results[k] = 0

2019-08-19 23:48:55,531 : MainThread : INFO : loading pre-existing wv from /Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/data/vectors/w2v_wv.vectors
2019-08-19 23:48:55,550 : MainThread : INFO : loading pre-existing wv from /Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/data/vectors/w2v_wv.vectors
2019-08-19 23:48:55,569 : MainThread : INFO : loading pre-existing wv from /Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/data/vectors/w2v_wv.vectors
2019-08-19 23:48:55,593 : MainThread : INFO : loading pre-existing wv from /Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/data/vectors/ft_wv.vectors
2019-08-19 23:48:55,607 : MainThread : INFO : loading pre-existing vocab from data/vectors/ft_vocab.vectors
2019-08-19 23:48:55,625 : MainThread : INFO : loa

time: 1.25 s


In [5]:
for year in tasks:
    df = sts_data[sts_data.task == year]
    sents_a = df.sent_a.values.tolist()
    sents_b = df.sent_b.values.tolist()
    assert len(sents_a) == len(sents_b)

    task_length = len(sents_a)
    sents_a.extend(sents_b)
    
    sents = [IndexedSentence(s, i) for i,s in enumerate(sents_a)]
    
    for k, m in models.items():
        if k.split("-")[-1] != year:
            continue
        
        m_type  = k.split("-")[0]
        emb_type = k.split("-")[1]
        
        m.train(sents)
        r = pearsonr(df.sim, compute_similarities(task_length, m))[0].round(4)
        
        results[f"{m_type}-{emb_type}-{year}"] = r * 100
        print(k, r)

2019-08-19 23:48:56,752 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-19 23:48:56,754 : MainThread : INFO : finished scanning 504 sentences with an average length of 13 and 6848 total words
2019-08-19 23:48:57,763 : MainThread : INFO : initializing sentence vectors for 504 sentences
2019-08-19 23:48:57,813 : MainThread : INFO : begin training
2019-08-19 23:48:58,150 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-19 23:48:58,151 : MainThread : INFO : training on 504 effective sentences with 5060 effective words took 0s with 1493 sentences/s
2019-08-19 23:48:58,182 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-19 23:48:58,185 : MainThread : INFO : finished scanning 504 sentences with an average length of 13 and 6848 total words


CBOW-W2V-2012 0.7467


2019-08-19 23:48:59,138 : MainThread : INFO : initializing sentence vectors for 504 sentences
2019-08-19 23:48:59,141 : MainThread : INFO : pre-computing SIF weights for 3000000 words
2019-08-19 23:49:02,114 : MainThread : INFO : begin training
2019-08-19 23:49:02,136 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-19 23:49:02,137 : MainThread : INFO : computing 1 principal components
2019-08-19 23:49:02,187 : MainThread : INFO : removing 1 principal components
2019-08-19 23:49:02,189 : MainThread : INFO : training on 504 effective sentences with 5060 effective words took 0s with 21632 sentences/s
2019-08-19 23:49:02,196 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-19 23:49:02,199 : MainThread : INFO : finished scanning 504 sentences with an average length of 13 and 6848 total words


SIF-W2V-2012 0.7874


2019-08-19 23:49:03,162 : MainThread : INFO : initializing sentence vectors for 504 sentences
2019-08-19 23:49:03,164 : MainThread : INFO : pre-computing uSIF weights for 3000000 words
2019-08-19 23:49:16,608 : MainThread : INFO : begin training
2019-08-19 23:49:16,632 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-19 23:49:16,633 : MainThread : INFO : computing 5 principal components
2019-08-19 23:49:16,648 : MainThread : INFO : training on 504 effective sentences with 5060 effective words took 0s with 20181 sentences/s
2019-08-19 23:49:16,654 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-19 23:49:16,657 : MainThread : INFO : finished scanning 504 sentences with an average length of 13 and 6848 total words


...

CBOW-FT-2017 0.5091


2019-08-19 23:52:06,203 : MainThread : INFO : initializing sentence vectors for 250 sentences
2019-08-19 23:52:06,205 : MainThread : INFO : pre-computing SIF weights for 2000000 words
2019-08-19 23:52:08,512 : MainThread : INFO : begin training
2019-08-19 23:52:08,538 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-19 23:52:08,538 : MainThread : INFO : computing 1 principal components
2019-08-19 23:52:08,542 : MainThread : INFO : removing 1 principal components
2019-08-19 23:52:08,543 : MainThread : INFO : training on 250 effective sentences with 2319 effective words took 0s with 9417 sentences/s
2019-08-19 23:52:08,548 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-19 23:52:08,549 : MainThread : INFO : finished scanning 250 sentences with an average length of 9 and 2319 total words


SIF-FT-2017 0.7487


2019-08-19 23:52:09,303 : MainThread : INFO : initializing sentence vectors for 250 sentences
2019-08-19 23:52:09,305 : MainThread : INFO : pre-computing uSIF weights for 2000000 words
2019-08-19 23:52:18,651 : MainThread : INFO : begin training
2019-08-19 23:52:18,678 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-19 23:52:18,678 : MainThread : INFO : computing 5 principal components
2019-08-19 23:52:18,684 : MainThread : INFO : training on 250 effective sentences with 2319 effective words took 0s with 8998 sentences/s


uSIF-FT-2017 0.6865
time: 3min 21s


In [6]:
pd.DataFrame.from_dict(results, orient="index", columns=["Pearson"])

,Pearson
CBOW-W2V-2012,74.67
SIF-W2V-2012,78.74
uSIF-W2V-2012,77.74
CBOW-FT-2012,50.99
SIF-FT-2012,79.10
uSIF-FT-2012,78.62
CBOW-W2V-2013,64.74
SIF-W2V-2013,67.41
uSIF-W2V-2013,69.72
CBOW-FT-2013,65.06


time: 34.4 ms
